In [1]:
##Need a way of opening the files.

import os
import time

import numpy as np
from spacy.lang import fr

DB1_DIR_PATH = "MACCROBAT2018"
DB2_DIR_PATH = "MACCROBAT2020"
#Open the db dirs.
DB_DIR1 = os.listdir(DB1_DIR_PATH)  #DIR 1
DB_DIR2 = os.listdir(DB2_DIR_PATH)  #DIR 2

DB_DIR1_LEN = len(DB_DIR1)
DB_DIR2_LEN = len(DB_DIR2)

print("There are " + str(DB_DIR1_LEN) + " files in DB1 directory")
print("There are " + str(DB_DIR2_LEN) + " files in DB2 directory")


def howManyPairsExistsInDB(dbPath):
    DB_PAIR_FILE_NUMBER = 0
    DB_DIR = os.listdir(dbPath)

    for filename in DB_DIR:
        if filename.endswith(".txt"):
            ann_file_path = os.path.join(dbPath, filename[:-4] + ".ann")
            if os.path.exists(ann_file_path):
                DB_PAIR_FILE_NUMBER += 1
    return DB_PAIR_FILE_NUMBER


DB1_PAIR_FILE_NUMBER = howManyPairsExistsInDB(DB1_DIR_PATH)
DB2_PAIR_FILE_NUMBER = howManyPairsExistsInDB(DB2_DIR_PATH)

print("There are " + str(DB1_PAIR_FILE_NUMBER) + " pairs in DB1 directory")

print("There are " + str(DB2_PAIR_FILE_NUMBER) + " pairs in DB2 directory")


#Ne intereseaza coloanele care incep cu T. Acolo se afla sign simpomele noastre in fisierele cu adnotari.

def cleanDB_addnotes(dbDir, DB_PATH):
    linesDeleted = 0
    linesInterestedIn = 0
    for filename in dbDir:  ##parcurgem lista de fisiere
        if filename.endswith(".ann"):  #daca e fisier de adnotare
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()
                with open(os.path.join(DB_PATH, filename), 'w') as fw:
                    for line in lines:  #parcurgem linie cu linie :D
                        if line.startswith("T"):
                            fw.write(line)
                            linesInterestedIn += 1
                        else:  #daca nu  incepe cu T vrem sa stergem:).
                            linesDeleted += 1
            except:
                print("Oops! something error :/")
    return linesDeleted, linesInterestedIn


lines_deleted, lines_interested = cleanDB_addnotes(DB_DIR1, DB1_DIR_PATH)

print("There are " + str(lines_deleted) + " lines deleted in DB1 directory")
print("There are " + str(lines_interested) + " lines interested in DB1 directory")

lines_deleted2, lines_interested2 = cleanDB_addnotes(DB_DIR2, DB2_DIR_PATH)

print("There are " + str(lines_deleted2) + " lines deleted in DB2 directory")
print("There are " + str(lines_interested2) + " lines interested in DB2 directory")

There are 400 files in DB1 directory
There are 400 files in DB2 directory
There are 200 pairs in DB1 directory
There are 200 pairs in DB2 directory
There are 34123 lines deleted in DB1 directory
There are 25041 lines interested in DB1 directory
There are 34449 lines deleted in DB2 directory
There are 25189 lines interested in DB2 directory


In [2]:
#Now that we have just the T addnoted we need to remain only with the SIGN SYMPTOM :)


def cleanDB_filterSIGN_SYMPTOM_adnots(DB_DIR, DB_PATH):
    linesDeleted = 0
    linesInterestedIn = 0
    for filename in DB_DIR:
        if filename.endswith(".ann"):
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()
                with open(os.path.join(DB_PATH, filename), 'w') as fw:
                    for line in lines:
                        word_line = line.strip().split()
                        if len(word_line) >= 2:
                            startline, label = word_line[0], word_line[1]
                        if label == "Sign_symptom":
                            fw.write(line)  #pastreaza linia 
                            linesInterestedIn += 1
                        else:
                            linesDeleted += 1
            except:
                print("Oops! something error :/")
    return linesDeleted, linesInterestedIn


line_del, line_inter = cleanDB_filterSIGN_SYMPTOM_adnots(DB_DIR1, DB1_DIR_PATH)
print("There are " + str(line_del) + " lines deleted in DB1 directory")
print("There are " + str(line_inter) + " lines interested in DB1 directory")

line_del2, line_inter2 = cleanDB_filterSIGN_SYMPTOM_adnots(DB_DIR2, DB2_DIR_PATH)

print("There are " + str(line_del2) + " lines deleted in DB2 directory")
print("There are " + str(line_inter2) + " lines interested in DB2 directory")


def cleanDB_eliminate_T_addnotes(DB_DIR, DB_PATH):
    for filename in DB_DIR:
        if filename.endswith(".ann"):
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()
                with open(os.path.join(DB_PATH, filename), 'w') as fw:
                    for line in lines:
                        words = line.strip().split(maxsplit=1)  # desparte primul cuvant
                        fw.write(words[1] + '\n')  # scrie linia inapoi dar omite acel cuvant care e de forma T###

            except:
                print("Oops! something error :/")


print("Eliminating unnecessary label in DB1 directory")
cleanDB_eliminate_T_addnotes(DB_DIR1, DB1_DIR_PATH)
print("Eliminating unnecessary label in DB2 directory")
cleanDB_eliminate_T_addnotes(DB_DIR2, DB2_DIR_PATH)
print("Eliminating successfully!")



There are 21682 lines deleted in DB1 directory
There are 3359 lines interested in DB1 directory
There are 21807 lines deleted in DB2 directory
There are 3382 lines interested in DB2 directory
Eliminating unnecessary label in DB1 directory
Eliminating unnecessary label in DB2 directory
Eliminating successfully!


In [3]:
#Now we are going to rename the labels. For the 1 word symptom we will rename them B_SYMPTOM
#we will have after that to deal with the composed symbols by many words.

#let's solve the easier part first.
#how we know it's of a 1 form symbol? it's formed of 4 words :----> LABEL START_POS END_POS [SYMPTOM]
def cleanDB_rename_oneWord_addnotes_(DB_DIR, DB_PATH):
    renamedLines = 0
    remainedLines = 0
    for filename in DB_DIR:
        if filename.endswith(".ann"):
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()
                with open(os.path.join(DB_PATH, filename), 'w') as fw:
                    for line in lines:
                        words = line.strip().split()  # desparte primul cuvant
                        if len(words) == 4:
                            fw.write("B_SYMPTOM" + ' ' + words[1] + ' ' + words[2] + ' ' + words[
                                3] + '\n')  #we write back the line modified
                            renamedLines += 1
                        else:
                            fw.write(line)  #we keep the more lengthy lines so we do not lose them:)
                            remainedLines += 1
            except:
                print("Oops! something error :/")
    return renamedLines, remainedLines


#now we rename the simple symptoms as they will be B_SYMPTOM and that's it .
renamedlines, remainedlines = cleanDB_rename_oneWord_addnotes_(DB_DIR1, DB1_DIR_PATH)

print("Renamed " + str(renamedlines) + " lines in DB1 directory")
print("Remained " + str(remainedlines) + " lines in DB1 directory")

renamedlines2, remainedlines2 = cleanDB_rename_oneWord_addnotes_(DB_DIR2, DB2_DIR_PATH)

print("Renamed " + str(renamedlines2) + " lines in DB2 directory")
print("Remained " + str(remainedlines2) + " lines in DB2 directory")



Renamed 2453 lines in DB1 directory
Remained 906 lines in DB1 directory
Renamed 2476 lines in DB2 directory
Remained 906 lines in DB2 directory


In [4]:
#now we have to deal with multiple words symptoms.

#multiple possibilities here :)


def cleanDB_rename_multiple_Words_addnotes(DB_DIR, DB_PATH):
    for filename in DB_DIR:
        if filename.endswith(".ann"):
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()
                with open(os.path.join(DB_PATH, filename), 'w') as fw:
                    for line in lines:
                        words = line.strip().split()  # we split the line into words
                        if len(words) > 4:  # if there are multiple words in the line symptom
                            current_poz = int(words[1])  #retin poz de inceput
                            fw.write(
                                "B_SYMPTOM" + ' ' + words[1] + ' ' + str(current_poz + len(words[3])) + ' ' + words[
                                    3] + '\n')  # Write first word that's the general rule.
                            current_poz += len(words[
                                                   3])  #actualizam faptul ca am trecut de primul cuvant (practic e end de 1st word acum)
                            current_poz += 1  #sarim un spatiu 

                            for word in words[4:]:
                                fw.write("I_SYMPTOM" + ' ' + str(current_poz) + ' ' + str(
                                    current_poz + len(word)) + ' ' + word + '\n')  # Write remaining words as I_SYMPTOM
                                current_poz += len(word)
                                current_poz += 1
                        else:
                            fw.write(line)  # Write lines with a single word as they are
            except Exception as e:
                print("Oops! something went wrong:", e)


# now we have on every line the simple symptom or the whole composed symptom 
#for example
# B_SYMPTOM 400 408 headache
# SIGN_SYMPTOM 500 509 acute pain
#before renaming into b_symptom and I_symptom we want to get the symptom in a list as a whole.
# that is why we build those list right here before breaking the symptoms into different tokens.

#build the symptom list based on the formatted annotated file
def getSYMPTOMS_LIST_FROM_DB(DB_DIR, DB_PATH):
    LIST_OF_SYMPTOMS = set()  # a list of unique elements
    for filename in DB_DIR:  #for every file
        if filename.endswith(".ann"):  #if its addnoted
            try:
                with open(os.path.join(DB_PATH, filename), 'r') as fr:
                    lines = fr.readlines()  ##we read all the lines of the file
                    #print("Succesfully read the lines!")
                    for line in lines:
                        words = line.lower().strip().split(maxsplit=3)  # we split the line into words and lower it

                        # print("Extracted words are :")
                        # print(words)

                        LIST_OF_SYMPTOMS.add(
                            words[3])  # here will be the words per se after the end label at this point  


            except Exception as e:
                print("Oops! something went wrong:", e)
    return LIST_OF_SYMPTOMS


#we want a unique list so we use SET in the function.(so no duplicates)
#first build the symptoms list for db1
print("Building list of symptoms from DB1 directory")
LIST_OF_SYMPTOMS1 = getSYMPTOMS_LIST_FROM_DB(DB_DIR1, DB1_DIR_PATH)

#now build fot the db2
print("Building list of symptoms from DB2 directory")
LIST_OF_SYMPTOMS2 = getSYMPTOMS_LIST_FROM_DB(DB_DIR2, DB2_DIR_PATH)

print("Successfully built lists of symptoms!")

#informative print about of our symptoms lists
print(f"Number of symptoms in DB1 directory: {len(LIST_OF_SYMPTOMS1)}")
print(f"Number of symptoms in DB2 directory: {len(LIST_OF_SYMPTOMS2)}")

#break the symptoms into multiple parts (B_SYMPTOM and I_SYMPTOM) for each db
cleanDB_rename_multiple_Words_addnotes(DB_DIR1, DB1_DIR_PATH)

print("Rename applied on multiple words in DB1 directory")

cleanDB_rename_multiple_Words_addnotes(DB_DIR2, DB2_DIR_PATH)

print("Rename applied on multiple words in DB2 directory")



Building list of symptoms from DB1 directory
Building list of symptoms from DB2 directory
Successfully built lists of symptoms!
Number of symptoms in DB1 directory: 1446
Number of symptoms in DB2 directory: 1456
Rename applied on multiple words in DB1 directory
Rename applied on multiple words in DB2 directory


In [5]:
#install different utilities we may need and add needed imports there
%pip install gensim
%pip install nltk
%pip install openpyxl

%pip install scipy==1.12 #needed so that nltk and gensim works ok.
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

/Users/mihaicotin/.zshenv:.:1: no such file or directory: /Users/mihaicotin/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/mihaicotin/.zshenv:.:1: no such file or directory: /Users/mihaicotin/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/mihaicotin/.zshenv:.:1: no such file or directory: /Users/mihaicotin/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/mihaicotin/.zshenv:.:1: no such file or directory: /Users/mihaicotin/.cargo/env
Note: you may need to restart the kernel to use updated packages.


In [6]:
#######STEP 1 ###########
# import pandas lib as pd
import pandas as pd  ##needed to read excel file in python

#We need now to compare our SYMPTOMS with the symptoms from the DIAGNOSTIC_BACKEND DB.
#Considering we have this db we will start building another set -->SYMPTOMS FROM THE DIAGNOSTIC and lets see how we can merge our databases into something better.

#the diagnostic db name
DB_DIAG_FILENAME = "diagnostic_db.xlsx"

## we need first to read the excel content once.

data_db = pd.read_excel(DB_DIAG_FILENAME)  #read the whole content


#now we will make a function that open that file and create the set of the symptoms they have.

def create_diagnostic_db_Set():
    DIAGNOSTIC_SYMPTOM_SET = set()
    for col in data_db.columns:
        DIAGNOSTIC_SYMPTOM_SET.add(col.lower())

    return DIAGNOSTIC_SYMPTOM_SET


DIAGNOSTIC_SYMPT_SET = create_diagnostic_db_Set()
print(f"There are {len(DIAGNOSTIC_SYMPT_SET)} Symptoms in the Diagnostic Database")


#now we want to see what are the exact symptoms we have in common :) we will get a set with those. as a reference

##since we have sets we may first check if the sets are identical or one is a subset of another ( we will get then 100% ratio if that's the case
def compare_databases_symtpom_exactForm(OUR_DB_SET, DIAG_SET):
    if OUR_DB_SET == DIAG_SET:
        print("The databases are equal")
        return 2
    else:
        if DIAG_SET <= OUR_DB_SET:
            print("The diagnosis database is a subset of ours")
            return -1
        if OUR_DB_SET <= DIAG_SET:
            print("The our database is a subset of the diagnosis database")
            return 1
        else:
            print("We cannot conclude anything")
            return 0


def compare_sets_databases_by_element(OUR_DB_SET, DIAG_SET):
    our_db_size = len(OUR_DB_SET)
    diag_db_size = len(DIAG_SET)
    COMMON_SYMPTOM_SET = set()

    for our_elem in OUR_DB_SET:
        for diag_elem in DIAG_SET:
            if our_elem == diag_elem:
                COMMON_SYMPTOM_SET.add(our_elem)

    return COMMON_SYMPTOM_SET


if compare_databases_symtpom_exactForm(LIST_OF_SYMPTOMS1, DIAGNOSTIC_SYMPT_SET) == 0:
    COMMON_SYMPTOM_SET1 = compare_sets_databases_by_element(LIST_OF_SYMPTOMS1, DIAGNOSTIC_SYMPT_SET)
    print("There are " + str(len(COMMON_SYMPTOM_SET1)) + " common symptoms found with db1!")

if compare_databases_symtpom_exactForm(LIST_OF_SYMPTOMS2, DIAGNOSTIC_SYMPT_SET) == 0:
    COMMON_SYMPTOM_SET2 = compare_sets_databases_by_element(LIST_OF_SYMPTOMS2, DIAGNOSTIC_SYMPT_SET)
    print("There are " + str(len(COMMON_SYMPTOM_SET2)) + " common symptoms found with db2!")

print("Success on the comparing...!\n\n")

print("The db1 set :" + str(COMMON_SYMPTOM_SET1) + "\n")
print("The db2 set :" + str(COMMON_SYMPTOM_SET2) + "\n")


## we see we have weak result so we may want to pull more from their diseases . lets see if their diseases are some of our symptoms...

def checkTheirDiseases():
    DISEASES_DIAGNOSTIC_SET = set()
    for rows in data_db.iterrows():
        col = data_db.columns[1]
        DISEASES_DIAGNOSTIC_SET.add((rows[1][col]).lower())

    return DISEASES_DIAGNOSTIC_SET


DISEASES_SET = checkTheirDiseases()
print("They have " + str(len(DISEASES_SET)) + " diseases!\n")

##now its time to recheck with this to see if we have something new
SYMPTOM_IN_COMMON_FROM_DISEASES1 = compare_sets_databases_by_element(LIST_OF_SYMPTOMS1, DISEASES_SET)
SYMPTOM_IN_COMMON_FROM_DISEASES2 = compare_sets_databases_by_element(LIST_OF_SYMPTOMS1, DISEASES_SET)

print("There are " + str(len(SYMPTOM_IN_COMMON_FROM_DISEASES1)) + " symptoms found with db1 from diseases!")
print("There are " + str(len(SYMPTOM_IN_COMMON_FROM_DISEASES2)) + " symptoms found with db2 from diseases!")

print("The db1 set from diseases: " + str(SYMPTOM_IN_COMMON_FROM_DISEASES1) + "\n")
print("The db2 set : from diseases: " + str(SYMPTOM_IN_COMMON_FROM_DISEASES2) + "\n")


## as we can see we get better result (2x!)) but very low.
##another thing we can do is to search our symptom as a substring to cover diagnostic laziness in cleaning their db

def build_common_set_with_diagnostic_db(OUR_DB_SET, DIAG_SET):
    COMMON_SYMPTOM_SET = set()
    for our_elem in OUR_DB_SET:
        for diag_elem in DIAG_SET:
            if our_elem in diag_elem:
                COMMON_SYMPTOM_SET.add(our_elem)

    return COMMON_SYMPTOM_SET


##retry our efforts
print("Building common set with diagnostic db from substring technique...\n")
FINAL_COMMON_SET1 = build_common_set_with_diagnostic_db(LIST_OF_SYMPTOMS1, DIAGNOSTIC_SYMPT_SET)
FINAL_COMMON_SET2 = build_common_set_with_diagnostic_db(LIST_OF_SYMPTOMS2, DIAGNOSTIC_SYMPT_SET)

print("The db1 set from their diagnostic symptoms with substring technique: " + str(len(FINAL_COMMON_SET1)) + "\n")
print("The db2 set from their diagnostic symptoms with substring technique: " + str(len(FINAL_COMMON_SET2)) + "\n")

print("The db1 set: " + str(FINAL_COMMON_SET1) + "\n")
print("The db2 set : " + str(FINAL_COMMON_SET2) + "\n")

## lets try on the diseases terms also for better results there
FINAL_DISEASES_SET1 = build_common_set_with_diagnostic_db(LIST_OF_SYMPTOMS1, DISEASES_SET)
FINAL_DISEASES_SET2 = build_common_set_with_diagnostic_db(LIST_OF_SYMPTOMS2, DISEASES_SET)

print("The db1 set from their diagnostic DISEASES with substring technique: " + str(len(FINAL_DISEASES_SET1)) + "\n")
print("The db2 set from their diagnostic DISEASES with substring technique: " + str(len(FINAL_DISEASES_SET2)) + "\n")

print("The db1 set from diseases " + str(FINAL_DISEASES_SET1) + "\n")
print("The db2 set from diseases : " + str(FINAL_DISEASES_SET2) + "\n")

print("DB1 FROM SYMPTOMS :\n")
print("BEFORE " + str(len(COMMON_SYMPTOM_SET1)) + "--- AFTER " + str(len(FINAL_COMMON_SET1)) + "\n")

print("DB2 FROM SYMPTOMS :\n")
print("BEFORE " + str(len(COMMON_SYMPTOM_SET2)) + "--- AFTER " + str(len(FINAL_COMMON_SET2)) + "\n")

print("DB1 FROM DISEASES :\n")
print("BEFORE " + str(len(SYMPTOM_IN_COMMON_FROM_DISEASES1)) + "--- AFTER " + str(len(FINAL_DISEASES_SET1)) + "\n")

print("DB2 FROM DISEASES :\n")
print("BEFORE " + str(len(SYMPTOM_IN_COMMON_FROM_DISEASES2)) + "--- AFTER " + str(len(FINAL_DISEASES_SET2)) + "\n")

print("Our similarities have increased by " + str(
    len(FINAL_COMMON_SET1) / len(COMMON_SYMPTOM_SET1)) + "x on the db1 with symptoms!")
print("Our similarities have increased by " + str(
    len(FINAL_COMMON_SET2) / len(COMMON_SYMPTOM_SET2)) + "x on the db2 with symptoms!")

print("Our similarities have increased by " + str(
    len(FINAL_DISEASES_SET1) / len(SYMPTOM_IN_COMMON_FROM_DISEASES1)) + "x on the db1 with diseases!")
print("Our similarities have increased by " + str(
    len(FINAL_DISEASES_SET2) / len(SYMPTOM_IN_COMMON_FROM_DISEASES2)) + "x on the db2 with diseases!")

print("Creating the final EXACT SET...")
FINAL_EXACT_SET = FINAL_DISEASES_SET1.copy()
FINAL_EXACT_SET.update(FINAL_DISEASES_SET2)
FINAL_EXACT_SET.update(FINAL_COMMON_SET1)
FINAL_EXACT_SET.update(FINAL_COMMON_SET2)
print("Final set length in common : " + str(len(FINAL_EXACT_SET)))
print("Their set length : " + str(len(DIAGNOSTIC_SYMPT_SET)))


There are 324 Symptoms in the Diagnostic Database
We cannot conclude anything
There are 50 common symptoms found with db1!
We cannot conclude anything
There are 50 common symptoms found with db2!
Success on the comparing...!


The db1 set :{'hemoptysis', 'paralysis', 'photophobia', 'cough', 'purpura', 'hypokinesia', 'overweight', 'hearing loss', 'dysphonia', 'seizures', 'dyskinesias', 'dizziness', 'pruritus', 'tinnitus', 'chills', 'hyperkinesis', 'hemianopsia', 'hematemesis', 'fatigue', 'jaundice', 'fever', 'headache', 'arthralgia', 'anorexia', 'polyuria', 'proteinuria', 'cyanosis', 'disease', 'confusion', 'vomiting', 'gait ataxia', 'nausea', 'hoarseness', 'diarrhea', 'tremor', 'dysarthria', 'syncope', 'oliguria', 'muscle weakness', 'pain', 'lethargy', 'hyperalgesia', 'neuralgia', 'edema', 'pallor', 'constipation', 'dyspnea', 'weight loss', 'dysuria', 'spasm'}

The db2 set :{'hemoptysis', 'paralysis', 'photophobia', 'cough', 'purpura', 'hypokinesia', 'overweight', 'hearing loss', 'dysp

In [7]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import numpy as np
import time
from gensim.models import Word2Vec

print("Diagnostic initial size " + str(len(DIAGNOSTIC_SYMPT_SET)))  ##324

print("\n\n\n")


def clean_set(set_to_clean):
    print("Set start size")
    print(len(set_to_clean))
    set_new = set()
    for word in set_to_clean:
        stripped_word = word.strip()
        split_array = stripped_word.split(",")
        set_new.add(split_array[0].strip().lower())
    print("Set end size")
    print(len(set_new))
    return set_new


LIST_OF_SYMPTOMS1 = clean_set(LIST_OF_SYMPTOMS1)
LIST_OF_SYMPTOMS2 = clean_set(LIST_OF_SYMPTOMS2)
DIAGNOSTIC_SYMPT_SET = clean_set(DIAGNOSTIC_SYMPT_SET)

first_10_elements = list(LIST_OF_SYMPTOMS1)[:10]
print("First 10 elements:", first_10_elements)

first_10_elements = list(DIAGNOSTIC_SYMPT_SET)[:10]
print("First 10 elements:", first_10_elements)

print("Our total words :" + str(len(LIST_OF_SYMPTOMS1)))
print("Diagnostic total words :" + str(len(DIAGNOSTIC_SYMPT_SET)))

#print("Words not in vocab ours : " + str(get_not_vocab_word(LIST_OF_SYMPTOMS1)))
#print("Words not in vocab diagnostic : " + str(get_not_vocab_word(DIAGNOSTIC_SYMPT_SET)))

print("Starting the similarity comparison!")
glove_input_file = "glove.42B.300d.txt"

# Path to save the converted GloVe vectors in Word2Vec format
word2vec_output_file = "glove.42B.300d.word2vec"

# Convert the GloVe vectors to Word2Vec format (one time you need that...!)
#print("Converting glove to word2vec")
#glove2word2vec(glove_input_file, word2vec_output_file)
#print("Converted word2vec to glove")

print("Getting the model!")
# Load the Word2Vec format GloVe model
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
vocabulary = glove_model.key_to_index  ## get the vocabulary

print("Got the model..continuing!")

result_addition = glove_model.most_similar(positive=["abdominal", "pain"], topn=1)
print("Result of addition:", result_addition)


def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0  # Return 0 if one of the vectors has zero norm
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity


#we need a way of getting all the single words symptom
def get_single_words(WORD_SET):
    SINGLE_WORDS = set()
    for word in WORD_SET:
        if len(word.strip().split()) == 1:
            SINGLE_WORDS.add(word)
    return SINGLE_WORDS


#we need a way to focus on the multiple words symptoms 
def get_composed_words(WORD_SET):
    SINGLE_WORDS = get_single_words(WORD_SET)
    MULTIPLE_WORDS = WORD_SET - SINGLE_WORDS
    return MULTIPLE_WORDS


## gives the corresponding sets we want
def get_similar_sets(SYMPTOM_SET_GIVEN):
    return get_single_words(SYMPTOM_SET_GIVEN), get_composed_words(SYMPTOM_SET_GIVEN)


## sunt format de 1 cuvant cu un cuvant
def compare_Single_Sets_Similarities(SYMPTOM_SET1, SYMPTOM_SET2):
    hashmap = {}  ##our hashmap
    OOV_OUR_SYMPTOMS = set()
    OOV_DIAGNOSTOC_SYMPTOMS = set()
    SINGLE_SIMILARITY_SET = set()
    word1_not_in_vocab = 0
    word2_not_in_vocab = 0

    threshold = 0.55
    for word1 in SYMPTOM_SET1:
        if word1 in vocabulary:
            hashmap[word1] = set()  #we init a set
            for word2 in SYMPTOM_SET2:
                if word2 in vocabulary:
                    similarity = cosine_similarity(glove_model[word1], glove_model[word2])
                    if similarity >= threshold:
                        ##SINGLE_SIMILARITY_SET.add(word1)  ##just once the other times will be ignored. our word
                        SINGLE_SIMILARITY_SET.add(word2)  ## add the similar word
                        hashmap[word1].add(word2)

                else:
                    OOV_DIAGNOSTOC_SYMPTOMS.add(word2)
                    word2_not_in_vocab += 1
        else:
            OOV_OUR_SYMPTOMS.add(word1)
            word1_not_in_vocab += 1

    print("Word1 not in vocabulary: " + str(word1_not_in_vocab))
    print("Word2 not in vocabulary: " + str(word2_not_in_vocab))
    return SINGLE_SIMILARITY_SET, hashmap, OOV_OUR_SYMPTOMS, OOV_DIAGNOSTOC_SYMPTOMS


def compare_similarities(SYMPTOM_SET1, SYMPTOM_SET2):
    SINGLE_SETWORD_1, MULTIPLE_SETWORD1 = get_similar_sets(SYMPTOM_SET1)
    SINGLE_SETWORD_2, MULTIPLE_SETWORD2 = get_similar_sets(SYMPTOM_SET2)
    SINGLE_SET, MAP_OF_WORDS, OOV_OURS_SYMP, OOV_DIAG_SYMP = compare_Single_Sets_Similarities(SINGLE_SETWORD_1,
                                                                                              SINGLE_SETWORD_2)
    print("Single word set similarity: " + str(len(SINGLE_SET)))  ###the similarity vector for 1 word to 1word
    print("Our oov number of words " + str(len(OOV_OURS_SYMP)))
    print("Diagnostic OOV number of words " + str(len(OOV_DIAG_SYMP)))

    #for key, values in MAP_OF_WORDS.items():
    #print(key, '->', values)
    return SINGLE_SETWORD_1, SINGLE_SETWORD_2, MULTIPLE_SETWORD1, MULTIPLE_SETWORD2, SINGLE_SET, MAP_OF_WORDS, OOV_OURS_SYMP, OOV_DIAG_SYMP


SET_OF_SINGLE_SYMPTOM1, SET_OF_SINGLE_SYMPTOM_DIAGNOSTIC, SET_OF_MULTIPLE_SYMPTOM1, SET_OF_MULTIPLE_SYMPTOM_DIAGNOSTIC, SIMILARITY_SET_SINGLEWORDED, MAP_OF_SIMILARITY, OOV_US, OOV_DIAGNOSIS = compare_similarities(
    LIST_OF_SYMPTOMS1, DIAGNOSTIC_SYMPT_SET)
##DB1_SIMILAR_SET_SYMTOM = get_set_based_on_similarity(LIST_OF_SYMPTOMS1, DIAGNOSTIC_SYMPT_SET)
##print(str(len(DB1_SIMILAR_SET_SYMTOM)) + "\n")
##print(DB1_SIMILAR_SET_SYMTOM)




Diagnostic initial size 324




Set start size
1446
Set end size
1446
Set start size
1456
Set end size
1456
Set start size
324
Set end size
281
First 10 elements: ['bullae', 'banded substances', 'structurally normal', 'contractions started', 'pruritus', 'compression', 'recurred', 'inversion', 'hemiparesis', '22q12 rearrangement']
First 10 elements: ['paralysis', 'deaf-blind disorders', 'infantile apparent life-threatening event', 'fetal hypoxia', 'albuminuria', 'torticollis', 'body weight', 'labor pain', 'dysphonia', 'aphonia']
Our total words :1446
Diagnostic total words :281
Starting the similarity comparison!
Getting the model!
Got the model..continuing!
Result of addition: [('discomfort', 0.7605615854263306)]
Word1 not in vocabulary: 71
Word2 not in vocabulary: 3240
Single word set similarity: 78
Our oov number of words 71
Diagnostic OOV number of words 5


In [37]:
def get_multipleWord_Set_encoding(SET_OF_WORDS, SET_OF_PHRASES):
    hashmap = {}
    map_of_encoding = {}
    encoding_context_map = {}
    SIMILARITY_SET = set()
    array_of_encoding = []

    vector_size = len(glove_model["example"])  # Accessing the size of a word vector
    encoding_of_word = np.zeros(vector_size)

    THRESHOLD = 0.70
    for word in SET_OF_PHRASES:
        words_from_phrase = str(word).strip().split()
        for subword in words_from_phrase:
            if subword in vocabulary:
                encoding_of_word = + glove_model[subword]  ## add the context
            else:
                encoding_of_word = np.zeros(vector_size)  ##clean
                break

        array_of_encoding.append(encoding_of_word)
        map_of_encoding[word] = encoding_of_word
        encoding_context_map[tuple(encoding_of_word)] = word  # Convert array to tuple

        encoding_of_word = np.zeros(vector_size)  ##clean

    for singleword in SET_OF_WORDS:
        if singleword in vocabulary:
            hashmap[singleword] = set()  # Use set instead of list to avoid repetitions
            for enc in array_of_encoding:
                if cosine_similarity(glove_model[singleword], enc) >= THRESHOLD:
                    hashmap[singleword].add(encoding_context_map[tuple(enc)])
                    #SIMILARITY_SET.add(singleword)
                    SIMILARITY_SET.add(encoding_context_map[tuple(enc)])

    print("Found " + str(len(SIMILARITY_SET)) + " similarity words!")
    return hashmap, SIMILARITY_SET, map_of_encoding, array_of_encoding


## in the last cell we compared word by word now we can word by phrase

map_word_to_multiple, similarity_set, map_of_encoding, array_of_encoding = get_multipleWord_Set_encoding(
    SET_OF_SINGLE_SYMPTOM1, SET_OF_MULTIPLE_SYMPTOM_DIAGNOSTIC)

for key, values in map_word_to_multiple.items():
    print(key, '->', values)

#next we still have our composed symptoms with their word (for that task we can use to do word-->multiple and inverse the map or 
#think to something specific
#we still have to compare phrase by phrase (our symptom composed with their)
####after those we need to merge all the hashsets into a bigger one and obtain a set with all the similarities we can keep to 
##know what symptom of us is needed and also what the diagnostic needs to remove.
#more tomorrow , 
#today 9.05.2024

print("oneWord set\n")
for key, values in MAP_OF_SIMILARITY.items():
    print(key, '->', values)

    ##we have so word-word, word-phrase comparison done
    ##now we have to implement phrase to word and phrase to phrase.


def get_similarity_phrase_to_word_Sets(SET_OF_PHRASES, SET_OF_WORDS):
    print("We got " + str(len(SET_OF_PHRASES)) + " phrases")
    print("We got " + str(len(SET_OF_WORDS)) + " words")

    hashmap = {}
    map_of_encoding2 = {}
    encoding_context_map = {}
    SIMILARITY_SET = set()
    array_of_encoding2 = []

    ## we want to compare each phrase to each word and do the corresponding mapping
    vector_size = len(glove_model["example"])  ##we want the size of the model vector for further calculus
    encoding_of_word = np.zeros(vector_size)
    THRESHOLD = 0.55

    for phrase in SET_OF_PHRASES:
        words_from_phrase = phrase.strip().split()  ##get the array of words
        for subword in words_from_phrase:
            if subword in vocabulary:
                encoding_of_word += glove_model[subword]
            else:
                encoding_of_word = np.zeros(vector_size)  ##reset the vector
                break

        hashmap[phrase] = set()
        array_of_encoding2.append(encoding_of_word)  ##save the encoding
        map_of_encoding2[phrase] = encoding_of_word
        encoding_context_map[tuple(encoding_of_word)] = phrase
        encoding_of_word = np.zeros(vector_size)

    print("We have " + str(len(array_of_encoding2)) + " encodings!");
    OOV_WORDS = 0
    for singleword in SET_OF_WORDS:
        if singleword in vocabulary:
            for encoding in array_of_encoding2:
                if cosine_similarity(glove_model[singleword], encoding) >= THRESHOLD:
                    hashmap[encoding_context_map[tuple(encoding)]].add(singleword)
                    SIMILARITY_SET.add(singleword)
                #SIMILARITY_SET.add(encoding_context_map[encoding])
        else:
            OOV_WORDS += 1
    print("OOV words from diagnostic: " + str(OOV_WORDS))
    print("Found " + str(len(SIMILARITY_SET)) + " similarity words from phrase to word!!")
    return hashmap, SIMILARITY_SET, map_of_encoding, array_of_encoding


phrase_to_word_hashmap, PHRASE_WORD_SIMILARITY_SET, PHRASE_ENC_ARRAY, PHRASES_ENCODING_ARR = get_similarity_phrase_to_word_Sets(
    SET_OF_MULTIPLE_SYMPTOM1, SET_OF_SINGLE_SYMPTOM_DIAGNOSTIC)

for key, values in phrase_to_word_hashmap.items():
    print(key, '->', values)


def phrase_to_phrase_Similarity_Set(SET_OF_PHRASES_US, SET_OF_PHRASES_DIAG):
    print("We got " + str(len(SET_OF_PHRASES_US)) + " phrases us")
    print("We got " + str(len(SET_OF_PHRASES_DIAG)) + " phrases diag")

    hashmap = {}
    map_of_encoding_us = {}
    encoding_context_map = {}
    SIMILARITY_SET = set()
    array_of_encoding_us = []

    ## we want to compare each phrase to each PHRASE and do the corresponding mapping
    vector_size = len(glove_model["example"])  ##we want the size of the model vector for further calculus
    encoding_of_word = np.zeros(vector_size)
    THRESHOLD = 0.70
    OOV_WORDS_US = 0

    ##we transform our phrases into single vector meaning
    for phrase in SET_OF_PHRASES_US:
        words_from_phrase = phrase.strip().split()  ##get the array of words
        for subword in words_from_phrase:
            if subword in vocabulary:
                encoding_of_word += glove_model[subword]
            else:
                encoding_of_word = np.zeros(vector_size)  ##reset the vector
                OOV_WORDS_US += 1
                break

        hashmap[phrase] = set()
        array_of_encoding_us.append(encoding_of_word)  ##save the encoding
        map_of_encoding_us[phrase] = encoding_of_word
        encoding_context_map[tuple(encoding_of_word)] = phrase
        encoding_of_word = np.zeros(vector_size)

    print("We have " + str(len(array_of_encoding_us)) + " encodings!");

    OOV_WORDS_DIAG = 0
    array_of_encoding_diag = []
    map_of_encoding_diag = {}
    encoding_context_map_diag = {}

    #we transform diagnostic phrases too.
    for phrase in SET_OF_PHRASES_DIAG:
        words_from_phrase = phrase.strip().split()  ##get the array of words
        for subword in words_from_phrase:
            if subword in vocabulary:
                encoding_of_word += glove_model[subword]
            else:
                encoding_of_word = np.zeros(vector_size)  ##reset the vector
                OOV_WORDS_DIAG += 1
                break
        array_of_encoding_diag.append(encoding_of_word)  ##save the encoding
        map_of_encoding_diag[phrase] = encoding_of_word
        encoding_context_map_diag[tuple(encoding_of_word)] = phrase
        encoding_of_word = np.zeros(vector_size)

    ##now we compare each codification
    for our_enc in array_of_encoding_us:
        for diag_enc in array_of_encoding_diag:
            if cosine_similarity(our_enc, diag_enc) >= THRESHOLD:
                hashmap[encoding_context_map[tuple(our_enc)]].add(encoding_context_map_diag[tuple(diag_enc)])
                #SIMILARITY_SET.add(encoding_context_map[tuple(our_enc)]) ##do we need our words there truly?
                SIMILARITY_SET.add(encoding_context_map_diag[tuple(diag_enc)])

    print("There are " + str(len(SIMILARITY_SET)) + " similarity words from phrase to phrase!")
    return hashmap, SIMILARITY_SET, encoding_context_map, encoding_context_map_diag


PHRASE_TO_PHRASE_MAP, SIMILARY_SET_PTP, OUR_PHRASE_ENC_MAP, DIAG_PHRASE_ENC_MAP = phrase_to_phrase_Similarity_Set(
    SET_OF_MULTIPLE_SYMPTOM1, SET_OF_MULTIPLE_SYMPTOM_DIAGNOSTIC)
####hash mare cu cele 4 unite
print("PHRASE TO PHRASE MAP!!!!\n\n\n")
for key, values in PHRASE_TO_PHRASE_MAP.items():
    if len(values) != 0:
        print(key, '->', values)











Found 20 similarity words!
diarrhoea -> {'postoperative nausea and vomiting'}
bezoar -> set()
bullae -> set()
impacted -> set()
infarction -> set()
fractures -> set()
modifications -> set()
pruritus -> set()
tumor -> set()
ulcers -> set()
compression -> set()
temperature -> set()
breathlessness -> set()
kyphosis -> set()
recurred -> set()
hyalinization -> set()
hemiparesis -> set()
inversion -> set()
sweating -> set()
contractions -> set()
enlarge -> set()
akinesia -> set()
cr -> set()
dissection -> set()
neutrophils -> set()
fevers -> set()
hypertrophied -> set()
hypersensitivity -> set()
blocking -> set()
calibre -> set()
hyponatremia -> set()
opacity -> set()
glycosuria -> set()
aminoaciduria -> set()
haematemesis -> set()
palpation -> set()
swellings -> set()
icterus -> set()
hematoma -> set()
swelling -> set()
adhesion -> set()
destruction -> set()
purpura -> set()
malnutrition -> set()
ectopic -> set()
polyp -> set()
hypogammaglobulinaemia -> set()
haematuria -> set()
comfortable